In [7]:
import pandas as pd
import numpy as np

In [64]:
col_list = ['Population', 'Violent_Crime', 'Murder', 'Rape', 'Rape_2', 'Robbery', 
            'Assault', 'Property_Crime', 'Burglary', 'Larceny', 'MV_Theft', 'Arson']

data = pd.read_excel('Colorado_2013_Crime.xls', names = col_list, header = 3, 
                     index_col = 0, skiprows = [0], skipfooter = 2).drop('Rape_2', 1)

In [65]:
data

,Population,Violent_Crime,Murder,Rape,Robbery,Assault,Property_Crime,Burglary,Larceny,MV_Theft,Arson
City,,,,,,,,,,,
Arvada,110792,163,0,33,34,96,2666,331,2138,197,22
Aspen,6685,14,0,5,0,9,314,17,286,11,0
Ault,1565,0,0,0,0,0,2,0,0,2,0
Aurora,343484,1436,23,224,468,721,10786,1981,7805,1000,80
Avon,6336,9,0,3,0,6,140,12,124,4,1
Basalt,3818,2,0,0,0,2,63,11,50,2,0
Berthoud,5316,3,0,1,1,1,45,13,28,4,0
Black Hawk,119,6,0,3,2,1,205,2,195,8,0
Blue River,854,1,0,0,1,0,4,0,4,0,0
